### 文本预处理

In [1]:
import numpy as np
import pandas as pd
import jieba
import os
import pickle
from tqdm import tqdm
from collections import Counter


In [2]:
# 导入原始数据
teatext_df = pd.read_csv('data/teapro.csv', encoding="utf-8")

In [3]:
# 数据观察与缺失值、无关列处理
teatext_df = teatext_df.drop(["no", "displayUserNick", "rateDate"], axis=1)
teatext_df_preprocessed = teatext_df.dropna(axis=0).reset_index(drop=True)

In [4]:
# 基于jieba的分词，以及停用词的删除
with open("data/stopWord.json", "r", encoding="utf-8") as f:
    stopWords = f.read().split("\n")
stopWords.append(' ')
stopWords.append(',')
def comment_cut(content):
    content_cut = list(jieba.cut(content.strip()))
    content_rm_stop_word = [word for word in content_cut if word not in stopWords]

    return content_rm_stop_word


teatext_df_preprocessed['rateContent'] = teatext_df_preprocessed['rateContent'].apply(comment_cut)

Building prefix dict from the default dictionary ...
Loading model from cache D:\Temporary1\jieba.cache
Loading model cost 0.715 seconds.
Prefix dict has been built successfully.


In [5]:
# 低频词删除
wordCount = {}
for c in teatext_df_preprocessed['rateContent']:
    for w in c:
        if w in wordCount:
            wordCount[w] += 1
        else:
            wordCount[w] = 1


def remove_low_freqen_word(c):
    return [w for w in c if wordCount[w] >= 2]


teatext_df_preprocessed['rateContent'] = teatext_df_preprocessed['rateContent'].apply(remove_low_freqen_word)

In [6]:
drop_index = []
for i in range(teatext_df_preprocessed.shape[0]):
    if teatext_df_preprocessed["rateContent"][i] == []:
        drop_index.append(i)
teatext_df_preprocessed = teatext_df_preprocessed.drop(drop_index)

In [7]:
with open('data/teatext_preprocessed.pkl', 'wb') as f:
    pickle.dump(teatext_df_preprocessed, f)
